In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
collins_duffy.csv
cosine_sim_feaure.gz
ec2-user@ec2-52-91-73-35.compute-1.amazonaws.com
sample.rtf
sample_submission.csv
semantic_pos_features.csv
stanfordData_train1.nlp
test.csv
test_with_cosine_v0.csv
train.csv
train_with_cosine_v0.csv



## Extract, Transform, Load

In [3]:
from sklearn.model_selection import train_test_split

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")

df_submit = read_data("input/test.csv")
df_submit = df_submit.reset_index(drop=True)
df_submit = df_submit.reset_index()
df_submit = df_submit.rename(columns = {'index': 'mat_idx'})


# Print the column names
print (df_train.columns.values.tolist())

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)
Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345796, 3)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate']


## Training the Model

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Read in the pre-built features for the training set
df_treekernel = pd.read_csv('input/collins_duffy.csv')
df_treekernel.drop('cdRaw',inplace=True,axis=1)
df_semantic = pd.read_csv('input/semantic_pos_features.csv')
df_semantic.drop('is_duplicate',inplace=True,axis=1)

df_train = df_train.merge(df_treekernel,how='inner',on='id')
df_train = df_train.merge(df_semantic,how='inner',on='id')

df_train.fillna(value=0, inplace=True)
    


In [5]:
df_train.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'cdNorm', 'S_sem_sim', 'T_sem_sim', 'sem_sim', 'S_unmatch_n',
       'T_unmatch_n', 'S_unmatch_n_p', 'T_unmatch_n_p', 'S_unmatch_a',
       'T_unmatch_a', 'S_unmatch_a_p', 'T_unmatch_a_p', 'S_unmatch_v',
       'T_unmatch_v', 'S_unmatch_v_p', 'T_unmatch_v_p', 'S_unmatch_pp',
       'T_unmatch_pp', 'S_unmatch_pp_p', 'T_unmatch_pp_p', 'S_unmatch_wp',
       'T_unmatch_wp', 'S_unmatch_wp_p', 'T_unmatch_wp_p'],
      dtype='object')

In [6]:
df_y = df_train['is_duplicate']
df_X = df_train[['cdNorm','S_sem_sim', 'T_sem_sim', 'sem_sim', 'S_unmatch_n',
       'T_unmatch_n', 'S_unmatch_n_p', 'T_unmatch_n_p', 'S_unmatch_a',
       'T_unmatch_a', 'S_unmatch_a_p', 'T_unmatch_a_p', 'S_unmatch_v',
       'T_unmatch_v', 'S_unmatch_v_p', 'T_unmatch_v_p', 'S_unmatch_pp',
       'T_unmatch_pp', 'S_unmatch_pp_p', 'T_unmatch_pp_p', 'S_unmatch_wp',
       'T_unmatch_wp', 'S_unmatch_wp_p', 'T_unmatch_wp_p']]

# df_X = df_train[['sem_sim']]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.20, random_state=37)

X_train_mat = X_train.values
y_train_mat = y_train.values

X_train_mat = MinMaxScaler().fit_transform(X_train_mat)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((323425, 24), (80857, 24), (323425,), (80857,))

## Cross-Validation

In [8]:
clf = LogisticRegression()
grid = {
    'C': [1e-6, 1e-3, 1e0, 1e1],
    'penalty': ['l1', 'l2']
}
cv = GridSearchCV(clf, grid, scoring='neg_log_loss', n_jobs=-1, verbose=1)
cv.fit(X_train_mat, y_train_mat)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [1e-06, 0.001, 1.0, 10.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=1)

In [9]:
# Print the best cross-validation parameters

print(cv.best_params_)
print(cv.best_estimator_.coef_)

{'C': 10.0, 'penalty': 'l1'}
[[ -0.5383624    6.05475281   5.77185526  -3.25860278  -7.6321311
  -14.3534604   -0.52205881  -0.37138842  -0.07864949   0.44651104
   -1.90401204  -2.30644438   1.7695238    1.46679158  -1.40937071
   -1.14875139  -1.43507039  -2.12006056   1.43109482   1.52176416
    0.20480198   0.53786458  -0.38013728  -0.92232064]]


## ROC

In [10]:
colors = ['r', 'g', 'b', 'y', 'k', 'c', 'm', 'brown', 'r']
lw = 1
Cs = [1e-6, 1e-4, 1e0]

plt.figure(figsize=(12,8))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for different classifiers')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

labels = []
for idx, C in enumerate(Cs):
    clf = LogisticRegression(C = C)
    clf.fit(X_train_mat, y_train_mat)
    print("C: {}, parameters {} and intercept {}".format(C, clf.coef_, clf.intercept_))
    fpr, tpr, _ = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=colors[idx])
    labels.append("C: {}, AUC = {}".format(C, np.round(roc_auc, 4)))

plt.legend(['random AUC = 0.5'] + labels)

C: 1e-06, parameters [[-0.00338033 -0.00050932 -0.00056271 -0.00041985 -0.00468066 -0.00218643
  -0.03459243 -0.03353999 -0.00328463 -0.00247894 -0.00154139 -0.00155418
  -0.00489672 -0.004443   -0.00221655 -0.00264609 -0.00117552 -0.00177553
  -0.00071771 -0.001168   -0.0022024  -0.00247938 -0.00076569 -0.00092812]] and intercept [-0.03700751]
C: 0.0001, parameters [[ 0.41448743  0.16415991  0.16127974  0.16389471 -0.13760283 -0.06253688
  -0.81467561 -0.75901177 -0.10161065 -0.0755538  -0.04423336 -0.04479697
  -0.10766817 -0.10354299 -0.03283144 -0.04563972 -0.02408921 -0.04511994
  -0.00667994 -0.01918056 -0.04865201 -0.04050418 -0.01223506 -0.01179639]] and intercept [-0.18740712]
C: 1.0, parameters [[ -0.54472791   4.60146008   4.52436342  -0.51227302  -7.49269469
  -12.12213358  -0.53242659  -0.46605011  -0.14106187   0.21026831
   -1.82440647  -2.01216426   1.67784425   1.23150424  -1.30159924
   -0.87771439  -1.26747227  -1.94783028   1.2816744    1.39932525
    0.19103912   0

## Precision-Recall

In [12]:
pr, re, _ = precision_recall_curve(y_test, cv.best_estimator_.predict_proba(X_test)[:,1])
plt.figure(figsize=(12,8))
plt.plot(re, pr)
plt.title('PR Curve (AUC {})'.format(auc(re, pr)))
plt.xlabel('Recall')
plt.ylabel('Precision')

## XGBoost

In [13]:
X_test_mat = MinMaxScaler().fit_transform(X_test.values)

import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)


/Users/Matthew/miniconda3/envs/bleedingEdge/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	train-logloss:0.687603	valid-logloss:0.687603
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.642547	valid-logloss:0.642568
[20]	train-logloss:0.610868	valid-logloss:0.610959
[30]	train-logloss:0.588001	valid-logloss:0.588224
[40]	train-logloss:0.57116	valid-logloss:0.571474
[50]	train-logloss:0.558264	valid-logloss:0.558681
[60]	train-logloss:0.548497	valid-logloss:0.549024
[70]	train-logloss:0.540947	valid-logloss:0.541591
[80]	train-logloss:0.535142	valid-logloss:0.535896
[90]	train-logloss:0.530656	valid-logloss:0.53148
[100]	train-logloss:0.527143	valid-logloss:0.528049
[110]	train-logloss:0.524366	valid-logloss:0.525352
[120]	train-logloss:0.522191	valid-logloss:0.523245
[130]	train-logloss:0.520419	valid-logloss:0.521543
[140]	train-logloss:0.518988	valid-logloss:0.520168
[150]	train-logloss:0.517789	valid-logloss:0.519032
[160]	train-logloss:0.516726	v

In [ ]:
retrained = cv.best_estimator_.fit(X_train_mat, y_train_mat)

probs = retrained.predict_proba(X_test_mat)

df_results = df_train.ix[X_test.index.values]
df_results['predictions'] = probs[:,1]

In [ ]:
compare = df_results[['question1','question2','is_duplicate','predictions']]
compare

In [105]:
from sklearn.metrics import log_loss

In [106]:
log_loss(df_results['is_duplicate'], df_results['predictions'])

0.56247646052292866

In [ ]:
df_features =  df_train['']

g = sns.pairplot(df_train[['cdNorm','S_sem_sim','is_duplicate']], hue="is_duplicate", diag_kind="hist")  
for ax in g.axes.flat:  
    plt.setp(ax.get_xticklabels(), rotation=45)

In [122]:
compare

,question1,question2,is_duplicate,predictions
92172,Which colors suits on dark complexion guys? Wh...,What color of pants should I wear with a dark ...,0,0.051735
334196,Why do so many people ask questions on Quora t...,Why do some people use Quora when a dictionary...,1,0.200971
392347,How are QR codes being used for inventory and ...,How small can a QR code be?,0,0.211468
4271,How is San Diego traffic?,How often do residents of San Diego go to Mexico?,0,0.114487
112185,What are the requisites to decide the banks fi...,Who is the/a person known to have foretold wit...,0,0.004479
218879,"If the selling price is doubled, the profit tr...",What percentage of domain speculators turn a p...,0,0.018139
92360,Is time an illusion created by man?,Does time exist?,0,0.199170
327319,How can I get over a spoiled friendship?,How do I get over a broken friendship?,1,0.614926
327238,Can I register my startup company in the US wi...,Can I start a company in the US without being ...,1,0.539178
341530,"What should I do as I’m 59-years-old, divorced...",How much average profits can we expect From E ...,0,0.011884
